# Introduction

The Fourier transform is one of the most widely-used mathematical tools in all of engineering. It is particularly powerful as an aid in convolving two functions. In our project, we want to investigate the differences between Fourier-assisted convolution and conventional iterative convolution in preprocessing images before k-means clustering and registration.

# Basic Theory: Fourier Series and Transforms

## Fourier Series

Just as a Taylor series tries to represent a portion of an arbitrary function as a sum of polynomial terms, the Fourier series tries to represent a portion of an arbitrary function as a sum of sinusoids.  In particular, the Fourier series represents a periodic function $g(t)$ over a period $T$ as a sum of sinusoids whose periods are equal to $T$ divided by some positive integer:

![realFS.png](screenshots/realFS.png)

Here $m$ and $n$ are the positive integers determining the periods for the cosine and sine terms. The coefficient $a_0$ is the average value of the function over the period $T$; this can be though of as a "DC bias" obtained by convolving $g(t)$ with a constant function $f(t)=1$. The other cosine and sine coefficients $a_m$ and $b_n$ are obtained by convolving $g(t)$ with a sine or cosine with the corresponding frequency:

$$\begin{split}
    a_0 &= \frac{1}{T}\int_0^T g(t) dt \\
    a_m &= \frac{2}{T}\int_0^T g(t) \cos\Big(\frac{2\pi mt}{T}\Big) dt \\
    b_n &= \frac{2}{T}\int_0^T g(t) \sin\Big(\frac{2\pi nt}{T}\Big) dt
\end{split}$$

Using Euler's formula ($e^{\theta i} = \cos \theta + i \sin \theta$) and some complex arithmetic, we can write the Fourier series more compactly using complex coefficients:

![compFourier.png](screenshots/compFourier.png)
![compcoeff.png](screenshots/compcoeff.png)

## Fourier Transform
The Fourier transform extends the basic idea of the Fourier series as a decomposition of a periodic function into component frequencies. The difference is that the Fourier transform allows for non-integer frequencies, turning the infinite sum over positive integers $n$ into an infinite _integral_ over $f$, the real-valued variable representing the component frequencies. Instead of a discrete list of complex coefficients $c_n$, we can obtain a function $G(f)$ representing the magnitude of the Fourier coefficient $G$ as a function of the real-valued frequency $f$. The function $G$ is called the _Fourier transform_ of $g$:

![fourierDefinition.png](screenshots/fourierDefinition.png)

We can also say that $g$ is the _inverse Fourier transform_ of $G$:

![inverse.jpg](screenshots/inverse.jpg)

Notice that aside from the variable of integration, the only difference between the FT and its inverse is the negative sign in the exponent.

## Multidimensional Fourier Transform

Fourier transforms can be performed in any number of dimensions. When moving to higher dimensions, it is necessary to integrate over each of the spatial dimensions. Also, each spatial dimension has a corresponding frequency variable in the Fourier domain. As an example, here are the formulas for the 2D Fourier transform and its inverse:

![2Dft.png](screenshots/2Dft.png)
![2Dift.png](screenshots/2Dift.png)

Here $x$ and $y$ replace $t$ as the spatial dimensions and $u$ and $v$ replace $f$ as the frequency variables. $u$ represents the frequencies of sinusoids propagating in the $x$ direction (wavefronts perpendicular to the $x$-axis); likewise, $v$ represents the spectrum of frequencies in the $y$ direction.


# Applications of FT to image processing

## FT as an aid to convolution

In image processing, we often need to convolve an image with a filter.  The Fourier transform has important implications as an alternative method for convolution. 

The convolution of two functions $g(t)$ and $h(t)$ is given by:

![convolution.png](screenshots/convolution.png)

A useful property of the Fourier transform is that convolution in the spatial or temporal domain ($x$ or $t$) is equivalent to simple multiplication in the Fourier/frequency domain ($f$):

![convolutionProperty.png](screenshots/convolutionProperty.png)


## Discrete Fourier Transform (DFT)

Before we can use FT to convolve images, we need a method to calculate Fourier transforms over discrete data sets such as the pixel values in a grayscale image. We also have to consider that the Fourier transform _itself_ must be defined over a discrete frequency domain, since we want to visualize the Fourier transform as another pixelated image.  Therefore, we can use the discrete-to-discrete Fourier transform and its inverse, i.e., one that is discrete both in the space domain and in the frequency domain:

![discFT.png](screenshots/discFT.png)


## Multidimensional DFT

One approach to a discrete-to-discrete multidimensional DFT is to convert the integrals back to sums as in the 1D DFT, and then sum over the rows and columns of the discrete spatial domain (for the FT) or the frequency domain (for the IFT):

![disc2Ddft.png](screenshots/disc2Ddft.png)
![disc2Dift.png](screenshots/disc2Dift.png)

However, I noticed that the DFT must sum over all elements in the domain regardless of its dimensionality, and that commutativity of addition means that the order in which terms are added does not matter. I wanted to test an approach that reuses the 1-dimensional DFT by flattening the 2D spatial domain into a 1D array of values, doing a 1D DFT on the resulting flattened array, and then reshaping the resulting 1D array of Fourier coefficients back into a 2D array.  I carried out this experiment below.

# Experiments and Results

## Testing the 2D-via-1D idea
I implemented a discrete, 1D DFT algorithm in Python (see method `compute_dft_complex` in `dft.py`). As a first experiment, I compared my 1D implementation to numpy's `np.fft.fft` method and got the same results every time, proving that my 1D DFT implementation is sound. Then, I tried to produce the same results as numpy's 2D FFT method `np.fft.fft2` using the folding-unfolding approach described above. 

![results/ftCheckerMine.png](results/ftCheckerMine.png)

![results/ftCheckerNumpy.png](results/ftCheckerNumpy.png)

![results/ftCheckerOrig.png](results/ftCheckerOrig.png)

![results/ftXmine.png](results/ftXmine.png)

![results/ftXnumpy.png](results/ftXnumpy.png)

![results/ftXorig.png](results/ftXorig.png)

![results/ftYmine.png](results/ftYmine.png)

![results/ftYnumpy.png](results/ftYnumpy.png)

![results/ftYorig.png](results/ftYorig.png)

The results were not perfect, but they were interesting. Somewhere in my algorithm, an axis flip occurred relative to the numpy method such that $x$-frequencies showed up on the $y$-axis and vice versa.  However, the magnitude of the frequencies was roughly the same, except that my code demonstrated an unexplained "spreading" phenomenon of the frequency peak along one axis or another.  The checkerboard input with its symmetry in $x$ and $y$ provided the most interesting example: my algorithm captured the approximate magnitude of the frequency, but not with the precision of `fft2`.

## Convolution with FT

The main experiment was testing whether FT could be used to convolve images with the same accuracy as iterative convolution, e.g. dragging a mean filter across an image. I implemented FT convolution in `ftFilter.py`. 

In the FT approach, I first padded the image and the filter with 0's until the matrices were the same size. I then FT'd both padded versions using `np.fft.fft2`, and multiplied the resulting FT matrices element by element. (See above for why multiplication in the Fourier domain is equivalent to convolution in the spatial domain.)  Finally, I inverse-FT'd the elementwise product matrix using 'np.fft.ifft2'. I compared the result with using the filtering program from Assignment 3. 

As a representative example, I convolved the checkerboard image with a 3-by-3 square of white pixels using the FT and compared the results with using a mean filter of size 3 in the Assignment 3 `filter.py` program:

![results/convCheckerFilter.png](results/convCheckerFilter.png)

![results/convCheckerMine.png](results/convCheckerMine.png)

Same basic motif, but mine is more colorful!

# References

Fourier Transform general info:
http://www.thefouriertransform.com/

Discrete 2D Fourier Transform:
http://fourier.eng.hmc.edu/e161/lectures/fourier/node9.html

1D DFT starter code:
https://www.nayuki.io/page/how-to-implement-the-discrete-fourier-transform

DFT in Image Processing:
http://www.dspguide.com/ch9/3.htm